In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Hypothyroidism/GSE151158'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptional analysis of non-fibrotic NAFLD progression"
!Series_summary	"Background & Aims:  Non-alcoholic steatohepatitis (NASH), a subtype of non-alcoholic fatty liver disease (NAFLD) that can lead to fibrosis, cirrhosis, and hepatocellular carcinoma, is characterized by hepatic inflammation.  Despite evolving therapies aimed to ameliorate inflammation in NASH, the transcriptional changes that lead to inflammation progression in NAFLD remain poorly understood.  The aim of this study is to define transcriptional changes in early, non-fibrotic NAFLD using a biopsy-proven non-fibrotic NAFLD cohort.  Methods:  We extracted RNA from liver tissue of 40 patients with biopsy-proven NAFLD based on NAFLD Activity Score (NAS) (23 with NAS ≤3, 17 with NAS ≥5) and 21 healthy controls and compared changes in expression of 594 genes involved in innate immune function.    Results:  Compared to healthy controls, NAFLD patients with NAS ≥5 had differential ex

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = True
trait_row = 12
age_row = 1
gender_row = 2

def convert_trait(value):
    if pd.isna(value):
        return None
    if 'hypothyroidism' in value:
        trait = value.split(': ')[1]
        if trait == 'Y':
            return 1
        elif trait == 'N':
            return 0
    return None

def convert_age(value):
    if pd.isna(value):
        return None
    if 'age' in value:
        age = value.split(': ')[1]
        try:
            return float(age)
        except ValueError:
            return None
    return None

def convert_gender(value):
    if pd.isna(value):
        return None
    if 'Sex' in value:
        gender = value.split(': ')[1]
        if gender == 'M':
            return 1
        elif gender == 'F':
            return 0
    return None

save_cohort_info('GSE151158', './preprocessed/Hypothyroidism/cohort_info.json', is_gene_available, trait_row is not None)

selected_clinical_data = geo_select_clinical_features(clinical_data, 'Hypothyroidism', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
csv_path = './preprocessed/Hypothyroidism/trait_data/GSE151158.csv'
selected_clinical_data.to_csv(csv_path)
print(preview_df(selected_clinical_data))


{'GSM4567420': [0.0, 53.0, 0.0], 'GSM4567421': [0.0, 40.0, 1.0], 'GSM4567422': [0.0, 51.0, 0.0], 'GSM4567423': [0.0, 36.0, 0.0], 'GSM4567424': [0.0, 44.0, 0.0], 'GSM4567425': [1.0, 60.0, 1.0], 'GSM4567426': [0.0, 31.0, 1.0], 'GSM4567427': [0.0, 41.0, 0.0], 'GSM4567428': [0.0, 55.0, 1.0], 'GSM4567429': [0.0, 15.0, 0.0], 'GSM4567430': [1.0, 57.0, 0.0], 'GSM4567431': [1.0, 56.0, 0.0], 'GSM4567432': [0.0, 34.0, 1.0], 'GSM4567433': [0.0, 43.0, 1.0], 'GSM4567434': [0.0, 49.0, 0.0], 'GSM4567435': [0.0, 55.0, 0.0], 'GSM4567436': [0.0, 52.0, 1.0], 'GSM4567437': [0.0, 35.0, 0.0], 'GSM4567438': [0.0, 35.0, 0.0], 'GSM4567439': [0.0, 40.0, 1.0], 'GSM4567440': [0.0, 34.0, 1.0], 'GSM4567441': [0.0, 42.0, 0.0], 'GSM4567442': [1.0, 53.0, 0.0], 'GSM4567443': [0.0, 33.0, 0.0], 'GSM4567444': [0.0, 31.0, 0.0], 'GSM4567445': [1.0, 57.0, 0.0], 'GSM4567446': [0.0, 42.0, 0.0], 'GSM4567447': [0.0, 48.0, 0.0], 'GSM4567448': [0.0, 47.0, 0.0], 'GSM4567449': [0.0, 51.0, 0.0], 'GSM4567450': [0.0, 65.0, 1.0], 'GSM456

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['ABCB1', 'ABCF1', 'ABL1', 'ADA', 'AHR', 'AICDA', 'AIRE', 'ALAS1', 'APP',
       'AREG', 'ARG1', 'ARG2', 'ARHGDIB', 'ATG10', 'ATG12', 'ATG16L1', 'ATG5',
       'ATG7', 'ATM', 'B2M'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = False


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))
    

# No code needed for STEP6 since requires_gene_mapping is False.


Gene annotation preview:
{'ID': ['ABCB1', 'ABL1', 'ADA', 'AHR', 'AICDA'], 'GB_ACC': ['NM_000927.3', 'NM_005157.3', 'NM_000022.2', 'NM_001621.3', 'NM_020661.1'], 'NS Probe ID': ['NM_000927.3:3910', 'NM_005157.3:3200', 'NM_000022.2:1300', 'NM_001621.3:1900', 'NM_020661.1:567'], 'SPOT_ID': [nan, nan, nan, nan, nan]}


### Step 7: Data Normalization and Merging

In [7]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Hypothyroidism/gene_data/GSE151158.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data are severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Hypothyroidism')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE151158', './preprocessed/Hypothyroidism/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Hypothyroidism/GSE151158.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Hypothyroidism', the least common label is '1.0' with 9 occurrences. This represents 14.75% of the dataset.
The distribution of the feature 'Hypothyroidism' in this dataset is fine.

Quartiles for 'Age':
  25%: 37.0
  50% (Median): 45.0
  75%: 52.0
Min: 15.0
Max: 65.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 25 occurrences. This represents 40.98% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

